In [1]:
import sys

In [2]:
class Node():
  def __init__(self,state,parent,action):
    self.state = state
    self.parent = parent
    self.action = action

In [3]:
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self,node):
        self.frontier.append(node)

    def contains_state(self,state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def array(self):
        return self.frontier

    def lenght(self):
        return len(self.frontier)

    def remove(self):
        if self.empty():
            raise Exception('empty frontier')
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
        return node

In [4]:
class QueueFrontier(StackFrontier):
  def remove(self):
    if self.empty():
      raise Exception('empty frontier')
    else:
      node = self.frontier[0]
      self.frontier = self.frontier[1:]
      return node

In [5]:
import math

def manhattan_distance(maze_value,heigh,width,goal):
    maze_distance = []
    for i in range (heigh):
        row = []
        for j in range (width):
            if(maze_value[i][j]) is False:
                x_distance = goal[0] - i
                y_distance = goal[1] - j
                row.append(abs(x_distance) + abs(y_distance))
            else:
                row.append('-')
        maze_distance.append(row)
    return maze_distance




In [17]:
def sort_by_distance(frontier,walls_distances):
    distances = []
    for i in range(frontier.lenght()):
        x,y = frontier.array()[i].state
        distances.append(walls_distances[x][y])
    #print("Arreglo de distancias")
    #print(distances)

    for i in range(frontier.lenght()-1):
        for j in range(i+1,frontier.lenght()):
            if(distances[i]>distances[j]):
                aux = frontier.array()[i]
                aux_num = distances[i]

                frontier.array()[i] = frontier.array()[j]
                distances[i] = distances[j]
                frontier.array()[j] = aux
                distances[j] = aux_num
    #print("Distancias ordenadas")
    #print(distances)
    #print()
    return frontier
#Priority Queue sort to use the manhattan distance


In [69]:
def cost(walls,heigh,width):
    path_cost = []
    for i in range(heigh):
        row = []
        for j in range(width):
            if not walls[i][j]:
                row.append(0)
            else:
                row.append('')
        path_cost.append(row)
    return path_cost

def increase_path_cost(child,walls_path_cost,path_cost,walls_distances):
    x,y = child.state

    walls_path_cost[x][y] += path_cost
    walls_distances[x][y] += walls_path_cost[x][y]

def update_path_cost(node,walls_path_cost):
    x,y = node.state
    cost = walls_path_cost[x][y]
    return cost+1



In [73]:
class Maze():
    def __init__(self,filename):
        with open(filename) as f:
            contents = f.read();

        if contents.count("A") != 1:
            raise Exception("Maze must have exactly one A")
        if contents.count("B")!= 1:
            raise Exception("Maze must have exactly one B")

        contents = contents.splitlines();
        self.heigh = len(contents)
        self.width = max(len(line) for line in contents)

        self.walls = []
        for i in range(self.heigh):
            row = []
            for j in range(self.width):
                try:
                    if(contents[i][j] == "A"):
                        self.start = (i,j)
                        row.append(False)
                    elif(contents[i][j] == "B"):
                        self.goal = (i,j)
                        row.append(False)
                    elif(contents[i][j] == " "):
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                        row.append(False)
            self.walls.append(row)

        self.walls_distances = manhattan_distance(self.walls,self.heigh,self.width,self.goal)

        self.path_cost = 1
        self.walls_path_cost = cost(self.walls,self.heigh,self.width)

        self.solution = None

    def print(self):
        solution = self.solution[1] if self.solution is not None else None

        for i,row in enumerate(self.walls):
            for j,col in enumerate(row):
                if col:
                    print("▓",end="")
                elif (i,j) == self.start:
                    print("A",end="")
                elif (i,j) ==self.goal:
                    print("B",end="")
                elif solution is not None and (i,j) in solution:
                    print("*",end="")
                else:
                    print(" ",end="")
            print()
        print()
        print(self.walls_distances)
        print(self.walls_path_cost)

    def neighbors(self,state):
        row,col = state
        candidates = [
            ("up", (row-1,col)),
            ("down", (row+1,col)),
            ("left", (row,col-1)),
            ("right", (row,col+1))
        ]
        result = []

        for action,(r,c) in candidates:
            if 0 <= r < self.heigh and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action,(r,c)))
        return result

    def solve(self):

        self.num_explored = 0

        start = Node(state=self.start, parent=None, action=None)
        frontier = QueueFrontier()
        frontier.add(start)

        self.explored = set()

        while True:

            if frontier.empty():
                raise Exception("No solution")

            node = frontier.remove()
            self.path_cost = update_path_cost(node,self.walls_path_cost)
            #update the path_cost

            self.num_explored += 1

            if node.state == self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions,cells)
                return

            self.explored.add(node.state)

            for action,state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    increase_path_cost(child,self.walls_path_cost,self.path_cost,self.walls_distances)
                    frontier.add(child)
            self.path_cost += 1
            frontier = sort_by_distance(frontier,self.walls_distances)
            #code to implement the Priority Queue

    def output_image(self, filename, show_solution=True, show_explored=False):
        from PIL import Image, ImageDraw
        cell_size = 50
        cell_border = 2

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.width * cell_size, self.heigh * cell_size),
            "black"
        )
        draw = ImageDraw.Draw(img)

        solution = self.solution[1] if self.solution is not None else None
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):

                # Walls
                if col:
                    fill = (40, 40, 40)

                # Start
                elif (i, j) == self.start:
                    fill = (255, 0, 0)

                # Goal
                elif (i, j) == self.goal:
                    fill = (0, 171, 28)

                # Solution
                elif solution is not None and show_solution and (i, j) in solution:
                    fill = (220, 235, 113)

                # Explored
                elif solution is not None and show_explored and (i, j) in self.explored:
                    fill = (212, 97, 85)

                # Empty cell
                else:
                    fill = (237, 240, 252)

                # Draw cell
                draw.rectangle(
                    ([(j * cell_size + cell_border, i * cell_size + cell_border),
                      ((j + 1) * cell_size - cell_border, (i + 1) * cell_size - cell_border)]),
                    fill=fill
                )

        img.save(filename)


In [18]:
m = Maze('maze1.txt')
m.print()
m.solve()

▓▓▓▓▓B▓
▓▓▓▓▓ ▓
▓▓▓▓  ▓
▓▓▓▓ ▓▓
     ▓▓
A▓▓▓▓▓▓

[['-', '-', '-', '-', '-', 0, '-'], ['-', '-', '-', '-', '-', 1, '-'], ['-', '-', '-', '-', 3, 2, '-'], ['-', '-', '-', '-', 4, '-', '-'], [9, 8, 7, 6, 5, '-', '-'], [10, '-', '-', '-', '-', '-', '-']]
1
2
3
4
5
6
7
8
9
10
11


In [74]:
m = Maze('maze5.txt')
m.print()
m.solve()
m.print()
m.output_image("maze.png", show_explored=True)

▓          B
▓ ▓▓▓▓▓▓▓▓▓ 
▓ ▓       ▓ 
▓ ▓ ▓▓▓▓▓ ▓ 
▓   ▓     ▓ 
▓▓▓ ▓ ▓▓▓▓▓ 
A   ▓       

[['-', 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0], ['-', 11, '-', '-', '-', '-', '-', '-', '-', '-', '-', 1], ['-', 12, '-', 10, 9, 8, 7, 6, 5, 4, '-', 2], ['-', 13, '-', 11, '-', '-', '-', '-', '-', 5, '-', 3], ['-', 14, 13, 12, '-', 10, 9, 8, 7, 6, '-', 4], ['-', '-', '-', 13, '-', 11, '-', '-', '-', '-', '-', 5], [17, 16, 15, 14, '-', 12, 11, 10, 9, 8, 7, 6]]
[['', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ['', 0, '', '', '', '', '', '', '', '', '', 0], ['', 0, '', 0, 0, 0, 0, 0, 0, 0, '', 0], ['', 0, '', 0, '', '', '', '', '', 0, '', 0], ['', 0, 0, 0, '', 0, 0, 0, 0, 0, '', 0], ['', '', '', 0, '', 0, '', '', '', '', '', 0], [0, 0, 0, 0, '', 0, 0, 0, 0, 0, 0, 0]]
▓**********B
▓*▓▓▓▓▓▓▓▓▓ 
▓*▓       ▓ 
▓*▓ ▓▓▓▓▓ ▓ 
▓***▓     ▓ 
▓▓▓*▓ ▓▓▓▓▓ 
A***▓       

[['-', 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21], ['-', 21, '-', '-', '-', '-', '-', '-', '-', '-', '-', 1], ['-', 21, '-', 17, 17, 17, 17, 17, 17, 17, '-',